In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import random
import pandas as pd
import numpy as np
from pathlib import Path

In [45]:
# fix seeds, make everything reproducible, etc (at least try to).
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

DATADIR = "../data"

MAX_WORDS = 4

In [46]:
# !pip install aicrowd-cli
%load_ext aicrowd.magic

The aicrowd.magic extension is already loaded. To reload it, use:
  %reload_ext aicrowd.magic


In [47]:
%aicrowd login

API Key valid
Gitlab oauth token invalid or absent.
It is highly recommended to simply run `aicrowd login` without passing the API Key.
Saved details successfully!


In [48]:
# https://github.com/explosion/spaCy/blob/657af5f91f88bb5a414ae133a99465dbc4f240be/spacy/lang/grc/stop_words.py
# we can use stopwords from spacy as a source of "known" short words.
# but the rules say that we should use only data from the dataset provided.

In [49]:
import re
if Path(DATADIR).exists():
  !rm -rf $DATADIR
!mkdir $DATADIR
%aicrowd ds dl -c htrec-2022 -o $DATADIR

train.csv:   0%|          | 0.00/395k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/45.5k [00:00<?, ?B/s]

In [50]:
import pywer
train_df = pd.read_csv( f"{DATADIR}/train.csv")
test_df = pd.read_csv(f"{DATADIR}/test.csv")

word_regex = re.compile("\W+")
word_regex2 = re.compile("(\W+)")

train_df.head()

,HUMAN_TRANSCRIPTION,SYSTEM_TRANSCRIPTION,CENTURY,IMAGE_PATH,TEXT_LINE_NUM
0,ἐγγινομένα πάθη μὴ σβεννύντες ἀλλὰ τῆ εκλύσει,ἐγγενομεναπαδημησμεννωτες ἀλλατῆε κλησει,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,1
1,τοῦ βίου τοῦ καθ ΄ εαυτοὺς πολλὰ γίνεσθαι συγχ...,του β ου του καλεαυτοὺς πολλαγινεσθαι συγχωρ όν,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,2
2,τες ἐμπυρίζουσι τὸν ἀμπελῶνα ἀλλὰ καὶ ὁ διὰ,τες εμπυριζου σιμαμπελῶνα ἀλλακαι ὅδξα,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,3
3,τῆς ἡδεῖας πλεονεξίας πολλοὺς εἰς τὴν τῶν ἀλλ,της ἐδίας πλσον ἐξιας πολλους ἐις τὴν τῶν ἀλ,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,4
4,οτρίων ἐπιθυμίαν προκαλούμενος ἐμπυρί,λοτρλων ἐπιθυμιαν προκαλουμένος ἐμπυρι,11,1 Bodleian-Library-MS-Barocci-102_00157_fol-75...,5


In [51]:
# !pip install nltk
# train_df.HUMAN_TRANSCRIPTION.apply()

In [52]:
for lib_dir in ["..", "../src"]:
    if not lib_dir in sys.path:
        sys.path.append(lib_dir)
from lm_utils import *

In [53]:
language_model = make_lm(train_df.HUMAN_TRANSCRIPTION.sum())

In [54]:
from tqdm.auto import tqdm
from datastruct import *
from common import *
from space_fixer import SpaceFixer

# for ht_line, mt_line in tqdm(train_df[["HUMAN_TRANSCRIPTION", "SYSTEM_TRANSCRIPTION"]].values[7:]):
#     ht_words = word_regex.split(ht_line)
#     mt_words = word_regex.split(mt_line)
#     #words_ = [remove_cap(word) for word in words]
#     #vocab.add_sentence(words)
#     dmatrix = build_path_matrix(mt_words, vocabs)
#     finished_paths = extract_paths(dmatrix)
#     for k in resplit_paths(finished_paths, mt_words):
#         variant = " ".join(k)
#         print(variant)
#     break


In [55]:
def extract_spaces_dict(x):
    result = dict()
    last_index = 0
    for w in word_regex2.split(x):
        if not word_regex.match(w):
            last_index += 1
        else:
            result[last_index] = w
    if not last_index in result:
        result[last_index] = ""
    return result
    

In [56]:

train_df["SYSTEM_TRANSCRIPTION_raw"] = train_df.SYSTEM_TRANSCRIPTION
train_df["SYSTEM_TRANSCRIPTION"] = train_df.SYSTEM_TRANSCRIPTION.apply(lambda x: lmr(x, lm=language_model))

ht_sequences_train = train_df.HUMAN_TRANSCRIPTION.apply(lambda x: word_regex.split(x)).values
mt_sequences_train = train_df.SYSTEM_TRANSCRIPTION.apply(lambda x: word_regex.split(x)).values
mt_spaces_train = train_df.SYSTEM_TRANSCRIPTION.apply(extract_spaces_dict).values

mt_texts_train = train_df.SYSTEM_TRANSCRIPTION.values
fixer = SpaceFixer(MAX_WORDS)
fixer.fill(ht_sequences_train)

  0%|          | 0/1875 [00:00<?, ?it/s]

In [57]:
words1 = [w for w in fixer.vocabs[1].get_words(1)]
words2 = [w for w in fixer.vocabs[1].get_words(2)]
stopwords = words1 + words2
    

In [91]:
def join_if_tuple(s):
    if isinstance(s, str):
        return s
    return " ".join(s)

corrected_texts = []
# mt_sequences_train = mt_sequences_train[-1:]
# mt_spaces_train = mt_spaces_train[-1:]
# mt_texts_train = mt_texts_train[-1:]
# ht_sequences_train = ht_sequences_train[-1:]
corrected_count = 0
k = 0
train_iter = zip(mt_sequences_train[k:], mt_spaces_train[k:], mt_texts_train[k:], ht_sequences_train[k:])
for i, (mt_words, line_spaces, mt_orig, ht_orig) in enumerate(tqdm(train_iter, total=len(mt_sequences_train))):
    # print(line_spaces)
    # print(mt_words)
    # mt_orig = " ".join(mt_words)
    best = mt_orig
    # temporary disable the following lines
    replacements = [
        [join_if_tuple(w) + s for w, s in zip(mt_split, spaces_after)]
        for mt_split, refs, spaces_after in fixer.split_words(mt_words, line_spaces, cutoff=3)
    ]
    replacements = ["".join(words) for words in replacements]

    #replacements = [
    #    ("".join([w + s for w, s in zip(mt_split, spaces_after)]), spaces_after) 
    #    
    #]
    # replacements, spaces_after = list(zip(*replacements))
    # replacements = list(replacements)
    if len(replacements) > 2:
        print(i, len(replacements))
        print(replacements)
        # print(spaces_after)
        print(line_spaces)
        # break
    
    best, is_corrected = lm_score(mt_orig, replacements, lm=language_model, return_corrected=True)
    if is_corrected:
        corrected_count += 1
        # break
    corrected_texts.append(best)
    # break

  0%|          | 0/1875 [00:00<?, ?it/s]

In [92]:
# mt_words, line_spaces, best, ht_orig
# class WordInfo:
#     def __init__(self, word, is_known=False, is_space=False):
#         self.word = word
#         self.is_known = is_known
#         self.is_space = is_space
#     def __repr__(self):
#         return f"{self.word}, is known={self.is_known}, is space={self.is_space}"
# info = []
# for w in mt_words:
#     dist = fixer.vocabs[1].find_closest(w, distance_only=True)
#     info.append(WordInfo(w, is_known = dist == 0))
# info


In [93]:
unicodedata.name("σ"), unicodedata.name("ς")
corrected_count
# fixer.split_matrix

554

In [94]:
# for mt_split, refs, spaces_after in fixer.resplit(mt_words, line_spaces):
#     print(mt_split, spaces_after, line_spaces)

# ht_orig, mt_orig
def postprocess_sigmas(sentence):
    def correct_word(word):
        if word_regex.match(word):
            return word
        #"ς"
        cw = []
        if len(word) < 1:
            return word
        for c in word[:-1]:
            name = unicodedata.name(c)
            new_char = c
            if name.find("FINAL SIGMA") >= 0:
                name = name.replace("FINAL ", "")
                try:
                    new_char = unicodedata.lookup(name)
                except:
                    new_char = c
            cw.append(new_char)
        c = word[-1]
        name = unicodedata.name(c)
        if name.find("SIGMA") >= 0 and name.find("FINAL SIGMA") < 0:
            name = name.replace("SIGMA", "FINAL SIGMA")
            try:
                new_char = unicodedata.lookup(name)
            except:
                new_char = c
        cw.append(c)
        return "".join(cw)

    words = [correct_word(x) for x in word_regex2.split(sentence)]
    return "".join(words)
    

# [postprocess_sigmas(t) for t in corrected_texts]

In [99]:
# print(ht_sequences_train[-1])
#print("mt:", mt_words)
#for k, v, spaces_after in fixer.split_words(mt_words, line_spaces):
#    print(k, v, spaces_after)

def get_short_words(texts, max_n=2):
    return set([w for line in texts for w in word_regex.split(line) if len(w) <= max_n and len(w) > 0])

VOWELS = set(["α", "ε", "η" "ι", "ο", "υ", "ω"])
def is_vowel(ch):
    return remove_cap(ch) in VOWELS
    
short_words = get_short_words(corrected_texts, max_n=1)

unknown_words = []
for w in short_words:
    word = Word(w)
    is_known = np.any([word.no_caps == w2.no_caps for w2 in words1])
    if not is_known:
        unknown_words.append(w)

unknown_words

['ς', 'ζ', 'λ', 'π', 'β']

In [100]:
# fixer.dmatrix
#fixer.split_matrix

In [101]:
from metrics import compute_metrics

In [133]:
ht_texts = train_df.HUMAN_TRANSCRIPTION.values
mt_texts = train_df.SYSTEM_TRANSCRIPTION_raw.values
ct = [lmr(t, word=" ς ", replacements=["ς ", " ", " σ ", " σ", " ὡς "], lm=language_model) for t in corrected_texts]
# ct2 = [postprocess_sigmas(t) for t in ct]
# ct2 = [lmr(t, word=" δ ", replacements=["δ ", " δ", " "], lm=language_model) for t in ct]
print("Unmodified:")
cerr_values_ht, cerr_values = compute_metrics(ht_texts, mt_texts, corrected_texts)
print("Corrected sigmas:")
cerr_values_ht1, cerr_values1 = compute_metrics(ht_texts, mt_texts, ct)
# print("corrected deltas:")
# cerr_values_ht2, cerr_values2 = compute_metrics(ht_texts, mt_texts, ct2)

Unmodified:
Candidate CER: 33.345936213460185
Candidate CERR: 0.9091387638324059
Corrected sigmas:
Candidate CER: 33.34449477201874
Candidate CERR: 0.9105802052738475


In [119]:

@dataclass
class WordToken:
    word: str
    is_word: bool
def resplit_text(text):
    return [
        WordToken(w, word_regex.match(w) is None) for w in word_regex2.split(text)
    ]

def make_variants(text):
    tokens_list = resplit_text(text)
    variants = []
    prefixes = []
    for i, wt in enumerate(tokens_list):
        if not wt.is_word and len(prefixes) == 0:
            prefixes = [[wt]]
            continue
        if not wt.is_word:
            continue
        new_prefixes = []
        if remove_cap(wt.word) in unknown_words:
            # operate
            if len(prefixes) == 0:
                if i < len(tokens_list) - 1:
                    next_token = [tokens_list[i + 1]]
                    new_prefixes.append([wt])
                else:
                    next_token = []
                new_prefixes.append([wt] + next_token)
                # new_prefixes.append(p)
            else:
                #next_token = tokens_list[i+1]
                for p in prefixes:
                    if i < len(tokens_list) - 1:
                        next_token = [tokens_list[i + 1]]
                        new_prefixes.append(p + [wt])
                    else:
                        next_token = []
                    if len(p) > 0:
                        new_prefixes.append(p[:-1] +[wt] + next_token)
                    # else:
                    #     new_prefixes.append([wt])
                    new_prefixes.append(p + [wt] + next_token)
                    new_prefixes.append(p)
            
        else:
            if i < len(tokens_list) - 1:
                next_token = [tokens_list[i + 1]]
            else:
                next_token = []
            # don't modify
            if len(prefixes) == 0:
                new_prefixes.append([wt] + next_token)
            else:
                # print(prefixes)
                new_prefixes.extend([prefix + [wt] + next_token for prefix in prefixes])
        prefixes = new_prefixes
    for prefix in prefixes:
        yield "".join([word.word for word in prefix])

corrected_texts2 = []
for text in corrected_texts:
    variants = list(make_variants(text))
    best = lm_score(text, variants, lm=language_model)
    corrected_texts2.append(best)


In [120]:
cerr_values_ht2, cerr_values2 = compute_metrics(ht_texts, mt_texts, corrected_texts2)


Candidate CER: 33.34485527959311
Candidate CERR: 0.9102196976994789


In [129]:
# corrected_texts2
# diff = cerr_values_ht - cerr_values
# ids = np.argsort(diff)
# for i in ids[:20]:
#     if corrected_texts[i] == mt_texts[i]:
#         continue
#     print(i, diff[i])
#     print(repr(corrected_texts[i]))
#     print("ht:", repr(ht_texts[i]))
#     print("mt:", repr(mt_texts[i]))
#     print("--")

In [130]:
# len(unknown_words)
# ct_mod = ct
# # unknown_words = "β ζ λ π ς".split()
# for w in unknown_words:
#     ct_mod = [
#         lmr(t, word=f" {w} ", replacements=[f"{w} ", f" {w}", " "], lm=language_model)
#         for t in ct_mod
#     ]
#     print("Word:", w)
#     cerr_values_ht3, cerr_values3 = compute_metrics(ht_texts, mt_texts, ct_mod)
#     print("--")
# # unknown_words


In [135]:
# diff_cer = (cerr_values_ht3 - cerr_values3)
# idx = np.argsort(diff_cer)
# for i in idx[:5]:
#     print(diff_cer[i])
#     print(repr(ct2[i]))
#     print(repr(ht_texts[i]), repr(mt_texts[i]))

In [136]:
test_df["SYSTEM_TRANSCRIPTION_raw"] = test_df.SYSTEM_TRANSCRIPTION
test_df["SYSTEM_TRANSCRIPTION"] = test_df.SYSTEM_TRANSCRIPTION.apply(lambda x: lmr(x, lm=language_model))

In [137]:
mt_orig_test = test_df.SYSTEM_TRANSCRIPTION.values
mt_sequences_test = test_df.SYSTEM_TRANSCRIPTION.apply(lambda x: word_regex.split(x)).values
mt_spaces_test = test_df.SYSTEM_TRANSCRIPTION.apply(extract_spaces_dict).values
corrected_texts_test = []
for i, (mt_words, line_spaces, mt_orig) in enumerate(tqdm(zip(mt_sequences_test, mt_spaces_test, mt_orig_test), total=len(mt_sequences_test))):
    # print(i)
    # print(mt_words)
    # mt_orig = " ".join(mt_words)
    # temporary disable the following lines
    best = mt_orig
    # replacements = ["".join([w+s for w, s in zip(mt_split, spaces_after)]) for mt_split, refs, spaces_after in fixer.split_words(mt_words, mt_spaces)]
    replacements = [
        [join_if_tuple(w) + s for w, s in zip(mt_split, spaces_after)]
        for mt_split, refs, spaces_after in fixer.split_words(mt_words, line_spaces, cutoff=3)
    ]
    replacements = ["".join(words) for words in replacements]
    best = lm_score(mt_orig, replacements, lm=language_model)
    corrected_texts_test.append(best)

  0%|          | 0/338 [00:00<?, ?it/s]

In [138]:
corrected_texts_test1 = [lmr(t, word=" ς ", replacements=["ς ", " ", " σ ", " σ", " ὡς "], lm=language_model) for t in corrected_texts_test]
ct_mod_test = corrected_texts_test1
for w in unknown_words:
    ct_mod = [
        lmr(t, word=f" {w} ", replacements=[f"{w} ", f" {w}", " "], lm=language_model)
        for t in ct_mod_test
    ]
# corrected_texts_test2 = [
#     lmr(t, word=" δ ", replacements=["δ ", " δ", " "], lm=language_model)
#     for t in corrected_texts_test1
# ]
# corrected_texts_test2 = [postprocess_sigmas(t) for t in corrected_texts_test1]

In [139]:
submission = pd.DataFrame(
    zip(test_df.IMAGE_PATH, ct_mod_test),
    columns=["ImageID", "Transcriptions"]
)
submission.sample()

,ImageID,Transcriptions
110,105 Bodleian-Library-MS-Barocci-59_00085_fol-4...,τὲ πρμον ην τὴν αν θησιν τὴν σρ


In [140]:
submission.to_csv("submission.csv", index=False)

In [141]:
%aicrowd submission create -c htrec-2022 -f submission.csv

An unexpected error occured!
HTTPSConnectionPool(host='www.aicrowd.com', port=443): Max retries exceeded with url: /api/v1/api_user (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fa22ac1c410>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
To get more information, you can run this command with -v.
To increase level of verbosity, you can go upto -vvvvv


In [28]:
#test_spaces = test_df.SYSTEM_TRANSCRIPTION.apply(lambda x: [w for w in word_regex2.split(x) if word_regex.match(w)]).values
#train_spaces1 = train_df.SYSTEM_TRANSCRIPTION.apply(lambda x: [w for w in word_regex2.split(x) if word_regex.match(w)]).values
#train_spaces2 = train_df.HUMAN_TRANSCRIPTION.apply(lambda x: [w for w in word_regex2.split(x) if word_regex.match(w)]).values

In [29]:
#from collections import Counter
#Counter([x for xs in test_spaces for xss in xs for x in xss])

In [30]:
#Counter([x for xs in train_spaces1 for xss in xs for x in xss])

In [31]:
# Counter([x for xs in train_spaces2 for xss in xs for x in xss])